In [2]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy.matlib
import numpy as np
import time
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import collections
sns.set_context('talk')
sns.set_style('white')
import os
import sys
import csv
import re

In [63]:
path2data='CHANGEHERE'

In [64]:
R1=pd.read_csv(path2data+'r1.fastq.gz',sep='\t',header=None)
R2=pd.read_csv(path2data+'r2.fastq.gz',sep='\t',header=None)

In [65]:
R1_seq=R1[1::4]

In [66]:
R2_seq=R2.ix[1::4]

In [67]:
R_combined=pd.concat([R1_seq,R2_seq],axis=1)

In [68]:
R_combined.columns=['R1-RNA','R2-Cell_barcode']

In [69]:
#check for primer sequence
checksubstr=['GGCACAAGCTTAATTAAGAATT' in x for x in R_combined['R1-RNA']]

In [70]:
R_matching=R_combined[checksubstr]

In [ ]:
R_matching['RNA-CBC']=[x+'-'+y for x,y in zip(R_matching['R1-RNA'],R_matching['R2-Cell_barcode'])]

In [72]:
R_matching.head()

,R1-RNA,R2-Cell_barcode,RNA-CBC
189,GGCAAACTGGGGCACAAGCTTAATTAAGAATTTTCNCTACNNGCTA...,AGAAACTTAT,GGCAAACTGGGGCACAAGCTTAATTAAGAATTTTCNCTACNNGCTA...
193,AGCAAACTGGGGCACAAGCTTAATTAAGAATTGCCAGGATNNACAA...,ATGCTGTCAC,AGCAAACTGGGGCACAAGCTTAATTAAGAATTGCCAGGATNNACAA...
245,AGCAAACTGGGGCACAAGCTTAATTAAGAATTCTAGACTCNNACAC...,ATGCTACCCA,AGCAAACTGGGGCACAAGCTTAATTAAGAATTCTAGACTCNNACAC...
273,AGCAAACTGGGGCACAAGCTTAATTAAGAATTGTTGACCTNNTCAG...,TAGCATGAGA,AGCAAACTGGGGCACAAGCTTAATTAAGAATTGTTGACCTNNTCAG...
285,AGCAAACTGGGGCACAAGCTTAATTAAGAATTAGGGCTTGNAGTGC...,TAACTACCCT,AGCAAACTGGGGCACAAGCTTAATTAAGAATTAGGGCTTGNAGTGC...


In [ ]:
R_matching['counts']=[1]*len(R_matching)

In [74]:
R_grouped=R_matching.groupby('RNA-CBC').sum()

In [75]:
R_grouped.sort_values('counts',ascending=False).head(20)

,counts
RNA-CBC,
AGCAAACTGGGGCACAAGCTTAATTAAGAATTCGAGCGGTCGGCACTCGGCCTAG-GGGGGGGGGG,13
AGCAAACTGGGGCACAAGCTTAATTAAGAATTCTCACAACAGGCCCCGGGCCTAG-GGGGGGGGGG,7
AGCAAACTGGGGCACAAGCTTAATTAAGAATTCTAACTCAGCGACTGGAGCCTAG-GGGGGGGGGG,6
AGCAAACTGGGGCACAAGCTTAATTAAGAATTTGGCGCGCAGATTTGCGGCCTAG-GGGGGGGGGG,6
AGCAAACTGGGGCACAAGCTTAATTAAGAATTGGTGTCGTTCCGAGAGCGCCTAG-GGGGGGGGGG,5
AGCAAACTGGGGCACAAGCTTAATTAAGAATTTCACTTATGGGTACGAGGCCTAG-GGGGGGGGGG,5
AGCAAACTGGGGCACAAGCTTAATTAAGAATTGGCTGAAGACCTTAAACGCCTAG-GGGGGGGGGG,4
AGCAAACTGGGGCACAAGCTTAATTAAGAATTCCCCTAAGGCACCAGCGCCCTAG-GGGGGGGGGG,4
AGCAAACTGGGGCACAAGCTTAATTAAGAATTCTCACCTTGAAGCGACCACCTAG-GGGGGGGGGG,4


Note we typically do observe a polyG in the cell barcode in some cases when we sequence on the NextSeq  (often the most abundant as seen here). However they are typically a small fraction of the entire run.

In [76]:
R_grouped.to_csv(path2data+'munched_counts.txt',sep='\t')